<a href="https://colab.research.google.com/github/Alishaw99/Neo4J_Projects/blob/main/RAG_With_Knowledge_graph(Neo4j).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# langchain-core

contains simple, core abstractions that have emerged as a standard, as well as LangChain Expression Language as a way to compose these components together. This package is now at version 0.1 and all breaking changes will be accompanied by a minor version bump.

# langchain-community
contains all third party integrations. We will work with partners on splitting key integrations out into standalone packages over the next month.

# langchain
contains higher-level and use-case specific chains, agents, and retrieval algorithms that are at the core of your application's cognitive architecture. We are targeting a launch of a stable 0.1 release for langchain in early January.#

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.7/293.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 29.3 MB/s eta 0:00:00
 

In [2]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [4]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [5]:
from typing import Tuple, List, Optional

In [6]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [7]:
from langchain_core.runnables import ConfigurableField

In [8]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


In [9]:
import os

In [10]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [11]:
from langchain_community.vectorstores import Neo4jVector

In [12]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [13]:
# NEO4J_URI="neo4j+s:/s.neo4j.io"
# NEO4J_USERNAME="neo4j"
# NEO4J_PASSWORD=""

In [14]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [15]:
from langchain_community.graphs import Neo4jGraph

In [16]:
graph = Neo4jGraph()

In [17]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Hazrat Umar").load()

In [18]:
len(raw_documents)

25

In [19]:
raw_documents[:3]

[Document(metadata={'title': 'Umar', 'summary': "Umar ibn al-Khattab (Arabic: عُمَر بْن ٱلْخَطَّاب, romanized: ʿUmar ibn al-Khaṭṭāb; c.\u2009582/583 – 644), also spelled Omar, was the second Rashidun caliph, ruling from August 634, when he succeeded Abu Bakr (r.\u2009632–634) as the second caliph, until his assassination in 644. Umar was a senior companion and father-in-law of the Islamic prophet Muhammad.\nUmar initially opposed Muhammad, his distant Qurayshite kinsman and later son-in-law. Following his conversion to Islam in 616, he became the first Muslim to openly pray at the Kaaba. Umar participated in almost all battles and expeditions under Muhammad, who bestowed the title al-Fārūq upon him, for his judgements. After Muhammad's death in June 632, Umar pledged allegiance to Abu Bakr (r.\u2009632–634) as the first caliph and served as the closest adviser to the latter until August 634, when the dying Abu Bakr nominated Umar as his successor.\nUnder Umar, the caliphate expanded at

In [20]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [21]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

In [22]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [23]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [24]:
graph_documents

[GraphDocument(nodes=[Node(id='Umar Ibn Al-Khattab', type='Person'), Node(id='Abu Bakr', type='Person'), Node(id='Muhammad', type='Person'), Node(id='Sasanian Empire', type='Empire'), Node(id='Byzantine Empire', type='Empire'), Node(id='Kaaba', type='Place'), Node(id='Persia', type='Country'), Node(id='Jerusalem', type='City'), Node(id="Abu Lu'Lu'A Firuz", type='Person')], relationships=[Relationship(source=Node(id='Umar Ibn Al-Khattab', type='Person'), target=Node(id='Abu Bakr', type='Person'), type='SUCCESSOR'), Relationship(source=Node(id='Umar Ibn Al-Khattab', type='Person'), target=Node(id='Muhammad', type='Person'), type='FATHER-IN-LAW'), Relationship(source=Node(id='Umar Ibn Al-Khattab', type='Person'), target=Node(id='Kaaba', type='Place'), type='PRAYED_AT'), Relationship(source=Node(id='Umar Ibn Al-Khattab', type='Person'), target=Node(id='Sasanian Empire', type='Empire'), type='CONQUERED'), Relationship(source=Node(id='Umar Ibn Al-Khattab', type='Person'), target=Node(id='Byz

In [25]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [26]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [27]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [28]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [29]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [30]:
from typing import Tuple, List, Optional

In [31]:
from langchain_community.vectorstores import Neo4jVector

In [32]:
from langchain_openai import OpenAIEmbeddings
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [33]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [34]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


In [35]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [36]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [37]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [39]:
entity_chain.invoke({"question": "Where was Umar born?"}).names

['Umar']

In [40]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [41]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [42]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [43]:
print(structured_retriever("Who is Omar I?"))

Umar Ibn Al-Khattab - SUCCESSOR -> Abu Bakr
Umar Ibn Al-Khattab - FATHER-IN-LAW -> Muhammad
Umar Ibn Al-Khattab - PRAYED_AT -> Kaaba
Umar Ibn Al-Khattab - CONQUERED -> Sasanian Empire
Umar Ibn Al-Khattab - CONQUERED -> Byzantine Empire
Umar Ibn Al-Khattab - CONQUERED -> Persia
Umar Ibn Al-Khattab - ALLOWED_ACCESS_TO -> Jerusalem
Umar Ibn Al-Khattab - ASSASSINATED -> Abu Lu'Lu'A Firuz
Umar Ibn Al-Khattab - ORDERED_INVASION -> Muslim Conquest Of Persia


In [44]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [45]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [46]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [47]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [48]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [49]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [50]:
prompt = ChatPromptTemplate.from_template(template)

In [51]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [53]:
chain.invoke({"question": "Which house did Omar I belong to?"})

Search query: Which house did Omar I belong to?


'Omar I belonged to the Banu Adi clan.'

In [54]:
chain.invoke(
    {
        "question": "When was he born?",
        "chat_history": [("Which house did Omar I belong to?", "House Of Tudor")],
    }
)

Search query: When was Omar I born?


'Umar Ibn Al-Khattab was born around 582/583.'